In [96]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
from torchsummary import summary
import numpy as np

In [97]:
df = pd.read_csv("https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv")

In [98]:
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [99]:
df['age'].value_counts()

,count
age,
18,69
19,68
46,29
52,29
50,29
47,29
48,29
51,29
45,29


In [100]:
columns = ['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges']

In [101]:
label_encoders = {}
for column in columns:
    label_encoders[column] = LabelEncoder()
    df[column] = label_encoders[column].fit_transform(df[column])

In [102]:
df

,age,sex,bmi,children,smoker,region,charges
0,1,0,197,0,1,3,1005
1,0,1,350,1,0,2,57
2,10,1,331,3,0,2,306
3,15,1,73,0,0,1,1097
4,14,1,223,0,0,1,254
...,...,...,...,...,...,...,...
1333,32,1,276,3,0,1,746
1334,0,0,302,0,0,0,118
1335,0,0,422,0,0,2,32
1336,3,0,146,0,0,3,91


In [103]:
X = df.drop('age', axis=1).values
y = df['age'].values
Y = y

In [104]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [105]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

In [106]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [107]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1070, 6), (268, 6), (1070,), (268,))

In [108]:
class CarEvaluationDense(nn.Module):
    def __init__(self):
        super(CarEvaluationDense, self).__init__()
        self.fc1 = nn.Linear(6, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the model, loss function, and optimizer
model = CarEvaluationDense()

In [109]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

In [110]:
model.train()
for epoch in range(50):
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(), y_batch.to()
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

Epoch 1, Loss: 528.2897
Epoch 2, Loss: 181.0416
Epoch 3, Loss: 115.5000
Epoch 4, Loss: 104.4932
Epoch 5, Loss: 100.6275
Epoch 6, Loss: 98.9334
Epoch 7, Loss: 96.7045
Epoch 8, Loss: 95.6002
Epoch 9, Loss: 93.8323
Epoch 10, Loss: 94.8835
Epoch 11, Loss: 93.2689
Epoch 12, Loss: 92.7859
Epoch 13, Loss: 93.4683
Epoch 14, Loss: 91.0174
Epoch 15, Loss: 89.5762
Epoch 16, Loss: 93.4853
Epoch 17, Loss: 90.3172
Epoch 18, Loss: 90.0944
Epoch 19, Loss: 88.2235
Epoch 20, Loss: 88.5812
Epoch 21, Loss: 86.4338
Epoch 22, Loss: 82.5647
Epoch 23, Loss: 83.6529
Epoch 24, Loss: 78.9191
Epoch 25, Loss: 76.8679
Epoch 26, Loss: 76.3795
Epoch 27, Loss: 75.1278
Epoch 28, Loss: 72.0345
Epoch 29, Loss: 75.1855
Epoch 30, Loss: 69.5626
Epoch 31, Loss: 68.9308
Epoch 32, Loss: 69.5558
Epoch 33, Loss: 68.1609
Epoch 34, Loss: 65.3982
Epoch 35, Loss: 64.7013
Epoch 36, Loss: 64.2902
Epoch 37, Loss: 63.9283
Epoch 38, Loss: 61.8371
Epoch 39, Loss: 62.9695
Epoch 40, Loss: 58.7586
Epoch 41, Loss: 58.5985
Epoch 42, Loss: 57.1

In [114]:
model.eval()
preds, actuals = [], []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to()
        outputs = model(X_batch).cpu().numpy()
        preds.extend(outputs)
        actuals.extend(y_batch.numpy())

mse = mean_squared_error(actuals, preds)
print(f"Test MSE: {mse:.4f}")

NameError: name 'mean_squared_error' is not defined